In [1]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [2]:
df = pd.read_csv('taxi_dataset_with_target.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [12]:
from sklearn.linear_model import LinearRegression

### Your code is here
model = LinearRegression()
model.fit(df.drop(['pickup_datetime'], axis=1), df['trip_duration'])

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [8]:
### Your code is here
coefficients = model.coef_
intercept = model.intercept_
coefficients, intercept

(array([198.46313675,   0.29631295,  56.46912165, 115.27353763]),
 171.65658145243458)

In [13]:
for column, coef in zip(X.columns, coefficients):
    print(column, coef)

print(model.intercept_)

vendor_id 198.4631367461269
passenger_count 0.2963129479546014
store_and_fwd_flag 56.4691216546317
distance_km 115.27353763048087
3.444711182964966e-11


In [1]:
### Полученные коэффициенты будут являться ответом на задание 5 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [25]:
import numpy as np

### Your code is here

def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
        """
        :param X: матрица объектов
        :param Y: вектор (матрица с 1 столбцом) ответов
        :param fit_intercept: добавить ли константную переменную в данные?
        
        :return: одномерный numpy-array с полученными коэффициентами beta
        """
        if fit_intercept:
            X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
        
        xxt = np.dot(X.T, X)
        xxt_inv = np.linalg.inv(xxt)
        xxt_inv_xxt = np.dot(xxt_inv, X.T)
        
        coef = np.dot(xxt_inv_xxt, Y)

        if fit_intercept:
            intercept = coef[0]
            coef = coef[1:]
        else:
            intercept = 0
    
        return coef, intercept


In [26]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

LinearRegressionByMatrix(X_train, Y_train)

(array([198.46313674,   0.29631295,  56.46912165, 115.27353763]),
 171.65658145459736)

In [5]:
### Полученные коэффициенты будут являться ответом на задание 6 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [28]:
### Your code is here
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
predictions = np.dot(X, coefficients) + intercept
predictions

array([4.55144433e-08, 4.55142688e-08, 4.55139908e-08, ...,
       4.55138575e-08, 4.55143348e-08, 4.55143309e-08])